In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import torch


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv
cuda


In [2]:
# data = pd.read_csv('/kaggle/input/titanic/gender_submission.csv')
data = pd.read_csv('/kaggle/input/titanic/train.csv')
# # data.head()
numerical_data_colums = ['PassengerId', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked', 'Survived']
# out_y = ['Survived']
data = data.loc[:, numerical_data_colums]
out_y = data.loc[:, ['PassengerId', 'Survived']]
out_y.head()
# data.head()

,PassengerId,Survived
0,1,0
1,2,1
2,3,1
3,4,1
4,5,0


features
* Pclass 1, 2, 3
* Sex 0, 1
* Age
* Fare

Convert sex and Emberked Column to numerical

removing the rows which have incomlete data entries

In [3]:
print(data.isnull().sum())
# data.dropna(inplace=True)
data.fillna(data.median(), inplace = True)
print(data.isnull().sum())
data

PassengerId      0
Pclass           0
Sex              0
Age            177
SibSp            0
Parch            0
Fare             0
Embarked         2
Survived         0
dtype: int64
PassengerId    0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Embarked       2
Survived       0
dtype: int64


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  This is separate from the ipykernel package so we can avoid doing imports until


,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Survived
0,1,3,male,22.0,1,0,7.2500,S,0
1,2,1,female,38.0,1,0,71.2833,C,1
2,3,3,female,26.0,0,0,7.9250,S,1
3,4,1,female,35.0,1,0,53.1000,S,1
4,5,3,male,35.0,0,0,8.0500,S,0
...,...,...,...,...,...,...,...,...,...
886,887,2,male,27.0,0,0,13.0000,S,0
887,888,1,female,19.0,0,0,30.0000,S,1
888,889,3,female,28.0,1,2,23.4500,S,0
889,890,1,male,26.0,0,0,30.0000,C,1


In [4]:
data_colums = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']

out_y = data.loc[:, 'Survived']
x = data.loc[:, data_colums]
out_y.head()

0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64

In [5]:
x.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,22.0,1,0,7.2500,S
1,1,female,38.0,1,0,71.2833,C
2,3,female,26.0,0,0,7.9250,S
3,1,female,35.0,1,0,53.1000,S
4,3,male,35.0,0,0,8.0500,S


Normalize age and fare and convert string data values to numerical

In [6]:

def sex_to_index(lst):
#     f = []
#     for i in lst:
#         if i == 'male':
#             f.append(1)
#         else:
#             f.append(2)
    if lst == 'male':
        return 1
    else:
        return 2

def embarked_to_index(i):
    if i == 'S':
        return 1
    elif i == 'C':
        return 2
    else:
        return 3
def normalize_age(age):
    mu = age.mean()
    sigma = age.std()
    return (age-mu)/sigma
def normalize_fare(fare):
    mu = fare.mean()
    sigma = fare.std()
    return (fare-mu)/sigma
        

In [7]:
x.head()
x['Sex'] = x['Sex'].apply(sex_to_index)
x['Embarked'] = x['Embarked'].apply(embarked_to_index)
x['Age'] = normalize_age(x['Age'])
x['Fare'] = normalize_fare(x['Fare'])
x.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,1,-0.565419,1,0,-0.502163,1
1,1,2,0.663488,1,0,0.786404,2
2,3,2,-0.258192,0,0,-0.488580,1
3,1,2,0.433068,1,0,0.420494,1
4,3,1,0.433068,0,0,-0.486064,1


In [8]:
np.shape(x)

(891, 7)

x -> features (7 colums)
out_y -> given output
w -> 7 x 1
ToDo: 
f = wx + b


z = sigmoid(f(x))


In [9]:
def sigmoid(val):
    val = np.exp(-val)
    val += 1
    val = 1/val
    return val
#signoid funciton

def activation(val):
    if val >= 0.5:
        return 1
    else:
        return 0

In [10]:
def f(w, x, b):
    t = np.dot(x, w) + b
    t = sigmoid(t)
#     t = np.round(t)
#     print(t)
    #     for i,_ in enumerate(t):
#         t[i] = activation(t[i])
    return t

training code

In [11]:
def train(w, x, b, y, alpha = 0.005):
    m = len(y)
    epsilon = 1e-6
    previos_err = 0
    iii= 0
    while True:
        h = f(w, x, b)
#         if(np.count_nonzero(h) != len(h)):
#             print('oops')
        J = np.sum(np.dot(y, np.log(h)) + np.dot(1-y, np.log(1-h)))
        J /= -m
        dj_dw = np.dot(x.T, h-y)
        dj_db = np.sum(h-y)
        w -= alpha * dj_dw / m
        b -= alpha * dj_db / m
        if iii % 100 == 0:
            print("Cost: " + str(J))
        if abs(previos_err-J) <= epsilon:
            break
        previos_err = J
        iii += 1
    return w, b

Training cell

In [12]:
w = np.ones(7)
w.reshape(7, 1)
    # np.shape(w)
b = 0
w, b = train(w, x, b, out_y, 0.095)
w, b

Cost: 3.533118020476578
Cost: 0.5275536650485417
Cost: 0.48017440573191994
Cost: 0.46278000185688584
Cost: 0.45413369475670695
Cost: 0.4495354310818955
Cost: 0.446957937909601
Cost: 0.4454248340796266
Cost: 0.4444539975617424
Cost: 0.4438001817247758
Cost: 0.4433339021805278
Cost: 0.4429841803861409
Cost: 0.4427106808671609
Cost: 0.4424896453545261
Cost: 0.44230654448912887
Cost: 0.44215212511710383
Cost: 0.44202023110759053
Cost: 0.4419065760173037


(array([-1.20505765,  2.58128966, -0.54238664, -0.32677411, -0.06473966,
         0.07069874,  0.20464911]),
 -1.424043535876054)

with 
aplha = 0.009, cost = 0.4464671762915648

aplha = 0.005, cost = 0.44978276046635995

aplha = 0.003, cost = 0.45192283819623147

aplha = 0.001, cost = 0.4615073691100471
clearly seen, decreasing alpha wont decrese cost always :)


so optimal alpha = 0.009

for this dataset

In [13]:
def predict(in_x):
    in_x = f(w, in_x, b)
    in_x = np.round(in_x)
    return in_x

Now, testing part

In [14]:
test_data = pd.read_csv('/kaggle/input/titanic/test.csv')
numerical_data_colums2 = ['PassengerId', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
test_data = test_data.loc[:,numerical_data_colums2]

print(test_data.isnull().sum())
# test_data.dropna(inplace=True)
test_data.fillna(test_data.median(), inplace = True)

print(test_data.isnull().sum())
# x.head()
test_data['Sex'] = test_data['Sex'].apply(sex_to_index)
test_data['Embarked'] = test_data['Embarked'].apply(embarked_to_index)
test_data['Age'] = normalize_age(test_data['Age'])
test_data['Fare'] = normalize_fare(test_data['Fare'])
print(test_data.head())

PassengerId     0
Pclass          0
Sex             0
Age            86
SibSp           0
Parch           0
Fare            1
Embarked        0
dtype: int64
PassengerId    0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Embarked       0
dtype: int64
   PassengerId  Pclass  Sex       Age  SibSp  Parch      Fare  Embarked
0          892       3    1  0.385769      0      0 -0.496818         3
1          893       3    2  1.369729      1      0 -0.511665         1
2          894       2    1  2.550481      0      0 -0.463545         3
3          895       3    1 -0.204607      0      0 -0.481898         1
4          896       3    2 -0.598191      1      1 -0.416992         1


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  import sys


In [15]:
in_x = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
in_x = test_data.loc[:, in_x]
test_out = predict(in_x)

In [16]:
passenger_id = list(test_data['PassengerId'])

test_out_lst = list(np.int_(test_out))
type(test_out_lst)
# passenger_id['Survived'] = test_out_lst
final_out = pd.DataFrame()
final_out['PassengerId'] = passenger_id
final_out['Survived'] = test_out_lst
final_out

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [17]:
final_out.to_csv('final_out.csv', index = False)

In [18]:
data['Age'].mean()

29.36158249158249